In [ ]:
pip install transformers torch tqdm pyngrok chromadb flask google-generativeai


In [35]:
import pandas as pd
df = pd.read_csv('/content/fatwa_data.csv')

print(df['question'][18])

ما هي الصلوات المفروضة؟ وما هي أوقاتها؟ وكم عدد ركعاتها؟


In [36]:
import re

def clean_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

def normalize_arabic(text):
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ـ', '', text)
    return text

def remove_diacritics(text):
    return re.sub(r'[\u064B-\u065F]', '', text)

def preprocess(text):
#    text = clean_whitespace(text)
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    return text


# Combine relevant fields for embedding
def prepare_doc(row):
    return f"العنوان: {clean_whitespace(row['fatwa title'])}\nالسؤال: {clean_whitespace(row['question'])}\nالإجابة: {clean_whitespace(row['answer'])}"

combined_text = []
combined_text = df.apply(prepare_doc, axis=1)
combined_text = combined_text.apply(preprocess)

print(combined_text[0])

العنوان: حكم اخذ الكتب والمقتنيات المهمله دون اذن اصحابها
السوال: هناك كتب واوراق كان اخوتي يستخدمونها في السنوات السابقه، وقد تركوها الان بالكامل في غرفه. فهل يجوز لي اخذها، رغم انهم لا يسمحون بذلك؟ انا ارغب في المحافظه عليها، لانني رايت اسم عبد الرحمن (اسم ابي) مكتوبا علي اغلب هذه الاوراق.
الاجابه: الحمد لله، والصلاه والسلام علي رسول الله، وعلي اله وصحبه، اما بعد:فهذه الكتب والاوراق ان كانت ملكا لاخوتك، ولا يسمحون لك باخذها، فليس لك اخذها دون اذنهم، ففي الحديث: لا يحل مال امري مسلم الا بطيب نفس منه. اخرجه احمد، والبيهقي، والدارقطني، وغيرهم.ووجود اسم ابيك علي صفحات بعضها، لا تاثير له في ذلك.والله اعلم.


In [37]:
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

# Load AraBERT
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabertv2")
# model.eval()

# Function to get embedding from CLS token
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Generate all embeddings
embeddings = [get_embedding(text) for text in tqdm(combined_text)]

# print(embeddings[0])

100%|██████████| 19/19 [00:21<00:00,  1.13s/it]


In [1]:
pip install chromadb

In [38]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_fatwa_db")
collection = client.get_or_create_collection("fatwa_rag")

collection.add(
    documents = combined_text.tolist(),
    embeddings = embeddings,
    ids=[str(i) for i in range(len(df))]
)

In [39]:
import google.generativeai as genai

genai.configure(api_key="****************")
# Change the model name to 'models/gemini-pro'
# or a supported model name and API version
gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')


In [40]:
def rag_query(user_question, top_k=3):
    # Embed user query
    query_embedding = get_embedding(user_question)

    # Retrieve top K similar fatwas
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    relevant_docs = results["documents"][0]

    # Prepare context
    context = "\n---\n".join(relevant_docs)

    # Prompt for Gemini
    prompt = f"""استنادًا إلى الفتاوى التالية، أجب عن السؤال التالي بدقة ووضوح.

الفتاوى:
{context}

السؤال:
{user_question}
"""


    response = gemini_model.generate_content(prompt)


    # Iterate through the response to get the full text
    full_text = ''
    for chunk in response:
        full_text += chunk.text

    return full_text

In [41]:
question = "عدد ركعات الفجر؟"
answer = rag_query(question)
print("الجواب:\n", answer)

الجواب:
 عدد ركعات صلاة الفجر ركعتان.  وهذا موضح في الفتوى الأولى.



In [31]:
pip install pyngrok

In [42]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok


# 🧱 Define Flask app
app = Flask(__name__)

html_template = """
<!DOCTYPE html>
<html>
<head>
  <title>Ask Gemini</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      background: linear-gradient(135deg, #f0f4f8, #d9e2ec);
      display: flex;
      flex-direction: column;
      align-items: center;
      justify-content: center;
      height: 100vh;
      margin: 0;
    }

    .container {
      background: white;
      padding: 40px;
      border-radius: 12px;
      box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1);
      width: 100%;
      max-width: 500px;
      text-align: center;
    }

    input[type="text"] {
      width: 100%;
      padding: 12px;
      font-size: 16px;
      border: 1px solid #ccc;
      border-radius: 8px;
      margin-bottom: 20px;
    }

    button {
      padding: 12px 24px;
      font-size: 16px;
      background-color: #007BFF;
      color: white;
      border: none;
      border-radius: 8px;
      cursor: pointer;
    }

    button:hover {
      background-color: #0056b3;
    }

    h2 {
      margin-bottom: 20px;
    }

    h3 {
      margin-top: 30px;
      color: #333;
    }

    p {
      background-color: #f7f7f7;
      padding: 15px;
      border-radius: 8px;
      white-space: pre-wrap;
    }
  </style>
</head>
<body>
  <div class="container">
    <h2>Ask a Question</h2>
    <form method="post">
      <input type="text" name="question" placeholder="Type your question here..." required>
      <button type="submit">Submit</button>
    </form>
    {% if answer %}
      <h3>Answer:</h3>
      <p>{{ answer }}</p>
    {% endif %}
  </div>
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def index():
    answer = ""
    if request.method == 'POST':
        question = request.form['question']
        try:
            answer = rag_query(question)
        except Exception as e:
            answer = f"❌ Error: {str(e)}"
    return render_template_string(html_template, answer=answer)

# Configure ngrok with your authtoken
ngrok.set_auth_token("2whXouub6NOEQanSH2fwkQ8ro0V_6g66ccufFGb2iCDfHdskx")

# 🔌 Start Flask with ngrok tunnel
public_url = ngrok.connect(5000)
print("🔗 Open your web app here:", public_url)

app.run(port=5000)


🔗 Open your web app here: NgrokTunnel: "https://f3e3-34-16-247-162.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/May/2025 12:25:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 12:25:11] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 12:25:19] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 12:25:32] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 12:25:43] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 12:26:08] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 12:42:18] "POST / HTTP/1.1" 200 -
